In [2]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import requests
import mercury
sns.set_theme(palette="tab10")

In [91]:
import duckdb

duckdb.execute("""
COPY (SELECT * FROM 'data/openalex/*.parquet') TO 'merge.parquet' (FORMAT 'parquet');
""")

In [3]:
part000 = pd.read_parquet("merge.parquet")
display(part000.head(2))

,id,doi,year,type,type_crossref,institutions,countries,primary_location,is_oa,coverage.last_state,coverage.last_error,coverage.last_error_data
0,https://openalex.org/W3103145119,https://doi.org/10.1038/s41592-019-0686-2,2020,article,journal-article,"[Universidade Federal de Minas Gerais, Univers...","[FR, NL, AU, JP, RU, CA, FI, BR, GB, CZ, EE, US]",Nature Methods,True,IN_FOSM_FR,MISMATCH_TYPE,"('journal-article', 'article')"
1,https://openalex.org/W2970684805,https://doi.org/10.1136/bmj.l4898,2019,article,journal-article,"[Cochrane, Monash University, Inserm, Harvard–...","[FR, AU, CA, GB, DK, US]",BMJ,True,IN_FOSM_FR,MISMATCH_TYPE,"('journal-article', 'article')"


In [27]:
bad_affiliations_df = part000[part000["coverage.last_error"] == "MISMATCH_FRENCH_AFFILIATION"]
display(bad_affiliations_df[["id", "doi", "coverage.last_error_data"]])
bad_affiliations = bad_affiliations_df["coverage.last_error_data"].to_list()

,id,doi,coverage.last_error_data
2049,https://openalex.org/W2805870922,https://doi.org/10.1681/asn.2017121260,"['Division of Nephrology and Hypertension, Cen..."
10381,https://openalex.org/W2889373178,https://doi.org/10.1039/c8nr05787g,['1015 Lausanne; EPFL; Laboratoire des Matéria...
11188,https://openalex.org/W2887463283,https://doi.org/10.12688/f1000research.14417.1,"['Clinique du Vertige, Centre Hospitalier Emil..."
11933,https://openalex.org/W4225140177,https://doi.org/10.1038/s41467-022-29959-1,"['Department of Sciences and Engineering, Sorb..."
15831,https://openalex.org/W3080555168,https://doi.org/10.1145/3407023.3409219,['Montimage']
17668,https://openalex.org/W4223896490,https://doi.org/10.1007/s00500-022-07068-x,"['Sorbonne Center of Artificial Intelligence, ..."
23124,https://openalex.org/W4224233561,https://doi.org/10.1002/anie.202203938,['Department of Sciences and Engineering Sorbo...
25604,https://openalex.org/W3101925414,https://doi.org/10.1039/c9na00323a,['Foundation of Research and Technology-Hellas...
26874,https://openalex.org/W2992815577,https://doi.org/10.1039/c9nr08453c,['EPFL; Faculty of Engineering; Institute of M...
27790,https://openalex.org/W4226275118,https://doi.org/10.1021/acs.cgd.2c00225,"['Department of Sciences and Engineering, Sorb..."


In [20]:
def clean_affiliations(affiliation):
    return affiliation.removeprefix("['").removesuffix("']").split("', '")

In [26]:
cleaned_affiliations = list(set(sum([clean_affiliations(affiliation) for affiliation in bad_affiliations], [])))
cleaned_affiliations

['Montimage',
 'Department of Sciences and Engineering Sorbonne University Abu Dhabi  38044 Abu Dhabi United Arab Emirates; Smart Materials Lab New York University Abu Dhabi  129188 Abu Dhabi United Arab Emirates',
 'EPFL; Faculty of Engineering; Institute of Materials; Laboratoire des Matériaux Semiconducteurs; École Polytechnique Fédérale de Lausanne',
 'Department of Sciences and Engineering, Sorbonne University Abu Dhabi, Abu Dhabi, UAE; Smart Materials Lab, New York University Abu Dhabi, Abu Dhabi, UAE',
 'Clinique du Vertige, Centre Hospitalier Emile Mayrisch, Esch-sur-Alzette',
 'Department of Sciences and Engineering, Sorbonne University Abu Dhabi, PO Box 38044, Abu Dhabi, UAE; Smart Materials Lab, New York University, Abu Dhabi, PO Box 38044, Abu Dhabi, UAE',
 'Department of Sciences and Engineering, Sorbonne University Abu Dhabi, PO Box 38044, Abu Dhabi, United Arab Emirates; Smart Materials Lab, New York University Abu Dhabi, PO Box 129188, Abu Dhabi, United Arab Emirates',


In [30]:
cleaned_affiliations[-1]

'Sorbonne Center of Artificial Intelligence, Sorbonne University-Abu Dhabi, Abu Dhabi, United Arab Emirates'

In [28]:
id = "W2889373178"
response = requests.get(f"https://api.openalex.org/works/{id}").json()
mercury.JSON(response)

In [93]:
# Build Sankey data
labels = ["OpenAlex", "UNDEFINED", "DOI_FOUND", "CRAWLED", "PARSED", "PARSED_FR", "IN_FOSM", "IN_FOSM_FR", 
          "BAD_DOI", "DOI_NO_ACCESS", "DOI_NO_CROSSREF", "DOI_NO_UNPAYWALL", "DOI_NO_PUBLICATION_YEAR",
          "DOI_EARLY_PUBLICATION_YEAR", "DOI_LATE_PUBLICATION_YEAR", "NOT_PARSED_FR", "NOT_PARSED", 
          "NOT_CRAWLED", "ALEX_DOI_NOT_FOUND", "ALEX_AUTHORSHIPS_NOT_FOUND", "ALEX_YEAR_NOT_FOUND", 
          "ALEX_TYPE_NOT_FOUND", "FOSM_YEAR_NOT_FOUND", "FOSM_TYPE_NOT_FOUND", "MISMATCH_YEAR", 
          "MISMATCH_TYPE", "MISMATCH_FRENCH_AFFILIATION", "OK"]

sources = []
targets = []
values = []


In [94]:
part000.loc[part000["coverage.last_error_data"] == "('journal-article', 'article')", "coverage.last_error"] = "OK"
part000.loc[part000["coverage.last_error"] == "OK", "coverage.last_error_data"] = None

In [95]:
states_dict = part000["coverage.last_state"].value_counts().to_dict()
print(states_dict)
for key in states_dict:
    sources.append(labels.index("OpenAlex"))
    targets.append(labels.index(key))
    values.append(states_dict.get(key))

print(sources, targets, values)
    


{'IN_FOSM_FR': 46324, 'PARSED': 496, 'PARSED_FR': 342, 'DOI_FOUND': 159, 'IN_FOSM': 24, 'UNDEFINED': 17, 'CRAWLED': 8}
[0, 0, 0, 0, 0, 0, 0] [7, 4, 5, 2, 6, 1, 3] [46324, 496, 342, 159, 24, 17, 8]


In [96]:
grouped_df = part000.groupby(by=["coverage.last_state", "coverage.last_error"], as_index=False).size()
display(grouped_df)
for index, row in grouped_df.iterrows():
    sources.append(labels.index(row["coverage.last_state"]))
    targets.append(labels.index(row["coverage.last_error"]))
    values.append(row["size"])

print(sources, targets, values)

,coverage.last_state,coverage.last_error,size
0,CRAWLED,NOT_PARSED,8
1,DOI_FOUND,DOI_LATE_PUBLICATION_YEAR,61
2,DOI_FOUND,DOI_NO_CROSSREF,9
3,DOI_FOUND,NOT_CRAWLED,89
4,IN_FOSM,MISMATCH_FRENCH_AFFILIATION,19
5,IN_FOSM,MISMATCH_TYPE,2
6,IN_FOSM,OK,3
7,IN_FOSM_FR,MISMATCH_TYPE,973
8,IN_FOSM_FR,MISMATCH_YEAR,26
9,IN_FOSM_FR,OK,45325


[0, 0, 0, 0, 0, 0, 0, 3, 2, 2, 2, 6, 6, 6, 7, 7, 7, 4, 5, 1] [7, 4, 5, 2, 6, 1, 3, 16, 14, 10, 17, 26, 25, 27, 25, 24, 27, 15, 27, 9] [46324, 496, 342, 159, 24, 17, 8, 8, 61, 9, 89, 19, 2, 3, 973, 26, 45325, 496, 342, 17]


In [97]:
grouped_df = part000.groupby(by=["coverage.last_error", "coverage.last_error_data"], as_index=False).size()
display(grouped_df)

,coverage.last_error,coverage.last_error_data,size
0,DOI_LATE_PUBLICATION_YEAR,"('publication_year', 2022)",11
1,DOI_LATE_PUBLICATION_YEAR,"('publication_year', 2023)",50
2,DOI_NO_ACCESS,"('doi_status_code', 404)",14
3,DOI_NO_ACCESS,"('doi_status_code', 500)",1
4,DOI_NO_ACCESS,"('doi_status_code', 502)",2
5,DOI_NO_CROSSREF,"('agency', 'datacite')",5
6,DOI_NO_CROSSREF,"('agency', 'medra')",2
7,DOI_NO_CROSSREF,"('agency', 'op')",2
8,MISMATCH_FRENCH_AFFILIATION,['1015 Lausanne; EPFL; Laboratoire des Matéria...,1
9,MISMATCH_FRENCH_AFFILIATION,"['Clinique du Vertige, Centre Hospitalier Emil...",1


In [98]:
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = labels,
      color = "blue"
    ),
    link = dict(
      source = sources,
      target = targets,
      value = values
  ))])

fig.update_layout(title_text="OpenAlex french publications coverage", font_size=10)
fig.show()